In [0]:
%pip install -U -qqqq mlflow databricks-langchain databricks-agents uv langgraph==0.3.4

In [0]:
dbutils.library.restartPython()

In [0]:
%load_ext autoreload
%autoreload 2

## Loggeamos el agente como un Mlflow Model

In [0]:
import mlflow
from mlflow.models.resources import (
    DatabricksServingEndpoint,
    DatabricksVectorSearchIndex,
    DatabricksFunction
)
from pkg_resources import get_distribution
from mlflow.types.agent import ChatAgentMessage
from databricks_langchain import ChatDatabricks
from agent import RAG_AGENT, LLM_ENDPOINT_NAME
import yaml

# Load configurations from YAML files
def load_config(config_file):
    with open(config_file, 'r') as f:
        return yaml.safe_load(f)

# Load configurations
rag_chain_config = load_config('rag_chain_config.yaml')
data_pipeline_config = load_config('data_pipeline_config.yaml')
destination_tables_config = load_config('destination_tables_config.yaml')

# Get Vector Search configuration

retriever_config = rag_chain_config.get('retriever_config')
VECTOR_SEARCH_INDEX = retriever_config.get('vector_search_index')
VECTOR_SEARCH_ENDPOINT = rag_chain_config.get('databricks_resources').get('vector_search_endpoint_name')
llm_endpoint_name = rag_chain_config.get('databricks_resources').get('llm_endpoint_name')
# Define resources needed by the agent
resources = [
    # LLM endpoint
    DatabricksServingEndpoint(endpoint_name=llm_endpoint_name),
    # Vector search index
    DatabricksVectorSearchIndex(
        index_name=VECTOR_SEARCH_INDEX
    ),
    # Python execution function
    DatabricksFunction(function_name="system.ai.python_exec")
]

# Set the registry URI to Unity Catalog
mlflow.set_registry_uri("databricks-uc")

# Define the model name in Unity Catalog
catalog_name = "theodore_kop_personal"
schema_name = "bcp"
model_name = "bcp_confluence_rag_agent"

full_model_name = f"{catalog_name}.{schema_name}.{model_name}"

with mlflow.start_run():
    # Log the model
    logged_agent_info = mlflow.pyfunc.log_model(
        artifact_path="agent",
        python_model="agent.py",
        extra_pip_requirements=[
            f"databricks-connect=={get_distribution('databricks-connect').version}",
            "databricks-sdk",
            "langchain",
            "langgraph"
        ],
        resources=resources,
        model_config=rag_chain_config,
        input_example={
            "messages": [
                {
                    "role": "user",
                    "content": "Qué es un EDV?",
                }
            ]
        }
    )
    
    # Register the model to Unity Catalog
    uc_model_info = mlflow.register_model(
        model_uri=logged_agent_info.model_uri,
        name=full_model_name
    )
    mlflow.log_dict(rag_chain_config, "agent_chain_config.json")

print(f"MLflow Run: {logged_agent_info.run_id}")
print(f"Model URI: {logged_agent_info.model_uri}")
print(f"Registered Model: {uc_model_info.name}")
print(f"Version: {uc_model_info.version}") 

## Test local

In [0]:
print("\nTesting the logged model...")

# Example conversation with notebook path in the query
messages = [
    ChatAgentMessage(
        role="user", 
        content="¿Que es un EDV?"
    )
]
# Get the response
response = RAG_AGENT.predict(messages=messages)
print("Agent Response:", response.messages[-1].content)

# Example of a follow-up question (no need to include notebook path again)
follow_up_messages = messages + [
    ChatAgentMessage(role="assistant", content=response.messages[-1].content),
    ChatAgentMessage(role="user", content="¿Como puedo realizar una solicitud de creación de uno nuevo?")
]

# Get the follow-up response
follow_up_response = RAG_AGENT.predict(messages=follow_up_messages)
print("\nFollow-up Response:", follow_up_response.messages[-1].content)

## Deploy agent

In [0]:
from databricks import agents
import time
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointStateReady, EndpointStateConfigUpdate

w = WorkspaceClient()

deployment_info = agents.deploy(model_name=full_model_name, model_version=uc_model_info.version)

browser_url = mlflow.utils.databricks_utils.get_browser_hostname()
print(f"\n\nView deployment status: https://{browser_url}/ml/endpoints/{deployment_info.endpoint_name}")

print("\nWaiting for endpoint to deploy.  This can take 15 - 20 minutes.", end="")
while w.serving_endpoints.get(deployment_info.endpoint_name).state.ready == EndpointStateReady.NOT_READY or w.serving_endpoints.get(deployment_info.endpoint_name).state.config_update == EndpointStateConfigUpdate.IN_PROGRESS:
    print(".", end="")
    time.sleep(30)